# Agent Memory

### Boilerplate code - llm initiation

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv("./env")

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.5-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

# openai_llm.invoke("Hello")

##### Adding multiple tools to an agent

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from IPython.display import display, Image
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage

def add(a:int, b:int) -> int:
    """
        Add two numbers a and b
        a: int
        b: int
    """
    return a + b

def divide(a:int, b:int) -> int:
    """
        Add two numbers a and b
        a: int
        b: int
    """
    return a / b

def subtract(a:int, b:int) -> int:
    """
        Subtract two numbers a and b
        a: int
        b: int
    """
    return a - b

tools = [add, subtract, divide]

llm_with_tools = google_llm.bind_tools(tools)
# llm_with_tools = openai_llm.bind_tools(tools)

def agent(state: MessagesState) -> MessagesState:
    return ({"messages": llm_with_tools.invoke(state.get("messages"))})


builder = StateGraph(MessagesState)
builder.add_node("agent", agent)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", "agent")

graph = builder.compile()

query = "Add 10 with 20, then divide it by 2 and subtract it by 5"
res = graph.invoke({"messages": HumanMessage(content=query)})

for msg in res.get('messages'):
    msg.pretty_print()

display(Image(graph.get_graph().draw_mermaid_png()))

##### If we don't have memory, conversation won't be persisted

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from IPython.display import display, Image
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

def add(a:int, b:int) -> int:
    """
        Add two numbers a and b
        a: int
        b: int
    """
    return a + b

def divide(a:int, b:int) -> int:
    """
        Add two numbers a and b
        a: int
        b: int
    """
    return a / b

def subtract(a:int, b:int) -> int:
    """
        Subtract two numbers a and b
        a: int
        b: int
    """
    return a - b

tools = [add, subtract, divide]

# llm_with_tools = google_llm.bind_tools(tools)
llm_with_tools = openai_llm.bind_tools(tools)

def agent(state: MessagesState) -> MessagesState:
    return ({"messages": llm_with_tools.invoke(state.get("messages"))})


builder = StateGraph(MessagesState)
builder.add_node("agent", agent)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", "agent")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

config = {"configurable":{"thread_id": "abc"}}

query_1 = "Add 10 with 20"
res = graph.invoke({"messages": HumanMessage(content=query_1)}, config=config)

# for msg in res.get('messages'):
#     msg.pretty_print()


query_2 = "divide it by 2"
res = graph.invoke({"messages": HumanMessage(content=query_2)}, config=config)

# for msg in res.get('messages'):
#     msg.pretty_print()


query_3 = "now subtract it by 5"
res = graph.invoke({"messages": HumanMessage(content=query_3)}, config=config)

for msg in res.get('messages'):
    msg.pretty_print()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import display, Image
from langgraph.checkpoint.memory import MemorySaver

def add(a:int, b:int) -> int:
    """
        This will add two numbers
        a: int
        b: int
    """
    return a + b

def subtract(a:int, b:int) -> int:
    """
        This will subtract two numbers
        a: int
        b: int
    """
    return a - b

def divide(a:int, b:int) -> int:
    """
        This will divide two numbers
        a: int
        b: int
    """
    return a / b


tools = [add, subtract, divide]

llm_with_tools = openai_llm.bind_tools(tools)

def agent(state: MessagesState):
    return {"messages": llm_with_tools.invoke(state.get("messages"))}


builder = StateGraph(MessagesState)

builder.add_node("agent", agent)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", "agent")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

config={"configurable":{"thread_id": "abc123"}}

graph.invoke({"messages": HumanMessage(content="Add 10 with 20")}, config=config)
graph.invoke({"messages": HumanMessage(content="Subtract it by 8")}, config=config)

res = graph.invoke({"messages": HumanMessage(content="Divide it by 2")}, config={"configurable":{"thread_id": "abc123"}})


for msg in res["messages"]:
    msg.pretty_print()


display(Image(graph.get_graph().draw_mermaid_png()))
